In [41]:
# mlp for multi-label classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

In [42]:
# from sklearn.model_selection import train_test_split    
df1 = pd.read_csv('main_dataset_kNN.csv')
df2 = pd.read_csv('drug_dict_onehot.csv')
df1.head()


,Unnamed: 0,gender,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,map,BMI,type.of.heart.failure,...,measured.bicarbonate,carboxyhemoglobin,oxygen.saturation,partial.oxygen.pressure,oxyhemoglobin,anion.gap,free.calcium,total.hemoglobin,GCS,ageCat
0,0,0.0,0.271429,0.333333,0.500000,0.480159,0.383562,0.428309,0.046006,1.0,...,0.368876,0.073375,0.944444,0.400000,0.945187,0.420988,0.428889,0.408730,1.0,0.666667
1,1,0.0,0.142857,0.363636,0.583333,0.595238,0.547945,0.569853,0.053564,1.0,...,0.385827,0.132075,0.880000,0.187234,0.878342,0.394444,0.440000,0.226190,1.0,0.533333
2,2,1.0,0.200000,0.353535,0.555556,0.476190,0.547945,0.514706,0.076992,1.0,...,0.496063,0.000000,0.973333,0.365957,0.981283,0.286111,0.440000,0.410714,1.0,0.800000
3,3,1.0,0.242857,0.651515,0.611111,0.484127,0.589041,0.540441,0.049828,1.0,...,0.372375,0.087002,0.917778,0.335225,0.918449,0.435802,0.473333,0.549603,1.0,0.666667
4,4,0.0,0.271429,0.505051,0.555556,0.376984,0.445205,0.413603,0.054523,1.0,...,0.435477,0.084906,0.965185,0.412057,0.968731,0.386265,0.473333,0.581349,1.0,0.266667


In [43]:
# df1 = df1.sort_values(by=['inpatient.number'], ascending=True)
# df1.reset_index(drop = True, inplace = True)
# df1.head()

In [44]:
df2 = df2.sort_values(by=['inpatient.number'], ascending=True)
df2.reset_index(drop = True, inplace = True)
df2.head()

,Unnamed: 0,inpatient.number,Atorvastatin calcium tablet,Benazepril hydrochloride tablet,Deslanoside injection,Digoxin tablet,Dobutamine hydrochloride injection,Furosemide injection,Furosemide tablet,Hydrochlorothiazide tablet,Isoprenaline Hydrochloride injection,Isosorbide Mononitrate Sustained Release tablet,Meglumine Adenosine Cyclophosphate for injection,Milrinone injection,Nitroglycerin injection,Shenfu injection,Spironolactone tablet,Torasemide tablet,Valsartan Dispersible tablet,sulfotanshinone sodium injection
0,0,722128,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0
1,1,723327,1,0,1,1,0,1,1,0,0,1,0,1,0,0,1,1,1,0
2,2,723617,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,3,724385,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0
4,4,725509,0,0,1,1,0,1,1,0,0,0,0,1,0,1,1,1,0,0


In [45]:
#pd.concat([df1['inpatient.number'],df2['inpatient.number']]).drop_duplicates(keep=False)

In [46]:
#df1['inpatient.number'].equals(df2['inpatient.number'])

In [47]:
df1.drop(['Unnamed: 0'],axis= 1,inplace=True)
df1.head()

,gender,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,map,BMI,type.of.heart.failure,NYHA.cardiac.function.classification,...,measured.bicarbonate,carboxyhemoglobin,oxygen.saturation,partial.oxygen.pressure,oxyhemoglobin,anion.gap,free.calcium,total.hemoglobin,GCS,ageCat
0,0.0,0.271429,0.333333,0.500000,0.480159,0.383562,0.428309,0.046006,1.0,1.0,...,0.368876,0.073375,0.944444,0.400000,0.945187,0.420988,0.428889,0.408730,1.0,0.666667
1,0.0,0.142857,0.363636,0.583333,0.595238,0.547945,0.569853,0.053564,1.0,1.0,...,0.385827,0.132075,0.880000,0.187234,0.878342,0.394444,0.440000,0.226190,1.0,0.533333
2,1.0,0.200000,0.353535,0.555556,0.476190,0.547945,0.514706,0.076992,1.0,0.5,...,0.496063,0.000000,0.973333,0.365957,0.981283,0.286111,0.440000,0.410714,1.0,0.800000
3,1.0,0.242857,0.651515,0.611111,0.484127,0.589041,0.540441,0.049828,1.0,0.5,...,0.372375,0.087002,0.917778,0.335225,0.918449,0.435802,0.473333,0.549603,1.0,0.666667
4,0.0,0.271429,0.505051,0.555556,0.376984,0.445205,0.413603,0.054523,1.0,0.5,...,0.435477,0.084906,0.965185,0.412057,0.968731,0.386265,0.473333,0.581349,1.0,0.266667


In [48]:
df2.drop(['inpatient.number','Unnamed: 0'],axis= 1,inplace=True)
df2.head()
len(df2.columns)

18

In [49]:
X = df1
y = df2


In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
len(X), len(y_train), len(y_test)

(1996, 1596, 400)

In [51]:
model = Sequential()
model.add(Dense(30, input_dim=144, activation='relu'))
model.add(Dense(30,  activation='relu'))
model.add(Dense(18, activation='sigmoid'))
model.compile(loss='mse', optimizer='adam')


In [52]:
model.fit(X_train, y_train, epochs = 100)


Epoch 1/100
50/50 [==============================] - 0s 1ms/step - loss: 0.2035
Epoch 2/100
50/50 [==============================] - 0s 2ms/step - loss: 0.1530
Epoch 3/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1497
Epoch 4/100
50/50 [==============================] - 0s 2ms/step - loss: 0.1489
Epoch 5/100
50/50 [==============================] - 0s 2ms/step - loss: 0.1482
Epoch 6/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1473
Epoch 7/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1462
Epoch 8/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1453
Epoch 9/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1446
Epoch 10/100
50/50 [==============================] - 0s 2ms/step - loss: 0.1440
Epoch 11/100
50/50 [==============================] - 0s 2ms/step - loss: 0.1431
Epoch 12/100
50/50 [==============================] - 0s 2ms/step - loss: 0.1423
Epoch 13/100
50/50 [=================

In [53]:
model.evaluate(X_test,y_test)

13/13 [==============================] - 0s 584us/step - loss: 0.1449


0.144942969083786

In [55]:
model.predict(X_test)

array([[0.2914623 , 0.21082169, 0.63948077, ..., 0.12061045, 0.11161348,
        0.21895519],
       [0.43684646, 0.27001575, 0.05384979, ..., 0.01954255, 0.09432852,
        0.5062439 ],
       [0.49055538, 0.31675184, 0.5058608 , ..., 0.05569389, 0.16373631,
        0.42740852],
       ...,
       [0.43312716, 0.03106374, 0.16507924, ..., 0.1312809 , 0.06577891,
        0.18006939],
       [0.6441552 , 0.11306942, 0.38466805, ..., 0.20240885, 0.18967918,
        0.45452607],
       [0.33985785, 0.24268267, 0.25469336, ..., 0.0165861 , 0.13550529,
        0.1610975 ]], dtype=float32)

In [56]:
y_test

,Atorvastatin calcium tablet,Benazepril hydrochloride tablet,Deslanoside injection,Digoxin tablet,Dobutamine hydrochloride injection,Furosemide injection,Furosemide tablet,Hydrochlorothiazide tablet,Isoprenaline Hydrochloride injection,Isosorbide Mononitrate Sustained Release tablet,Meglumine Adenosine Cyclophosphate for injection,Milrinone injection,Nitroglycerin injection,Shenfu injection,Spironolactone tablet,Torasemide tablet,Valsartan Dispersible tablet,sulfotanshinone sodium injection
889,0,1,0,0,0,1,1,0,0,0,1,1,0,1,1,0,0,0
1674,1,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1
414,1,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,1,1
1427,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,1,0
849,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,0,0,1,0,0,1,1,1,0,0,0,1,0,0,1,0,0,0
576,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
963,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0
240,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,1
